In [0]:
# when using colab for the first time for this project, uncomment and run this cell
#!pip install autokeras
#!pip install tensorlow==2.1.0

In [0]:
%tensorflow_version 2.1

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.1`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [0]:
import numpy as np
import os
from os.path import isfile
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Dropout, Activation, GRU
from tensorflow.keras.layers import Conv2D, concatenate, MaxPooling2D, Flatten, Embedding, Lambda


from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K
from keras.utils import np_utils
from tensorflow.keras.optimizers import Adam, RMSprop

from tensorflow.keras import regularizers
import autokeras as ak



import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
dict_genres = {'Electronic':0, 'Experimental':1, 'Folk':2, 'Hip-Hop':3, 
               'Instrumental':4,'International':5, 'Pop' :6, 'Rock': 7  }


reverse_map = {v: k for k, v in dict_genres.items()}
print(reverse_map)

{0: 'Electronic', 1: 'Experimental', 2: 'Folk', 3: 'Hip-Hop', 4: 'Instrumental', 5: 'International', 6: 'Pop', 7: 'Rock'}


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls "/content/gdrive/My Drive/Music-Genre-Classifier"

 CNN-RNN_parallel.ipynb   preprocess.py  'Test Set'	 'Validation Set'
 Models			  README.md	 'Training Set'


In [0]:
npzfile = np.load('/content/gdrive/My Drive/Music-Genre-Classifier/Training Set/shuffled_train.npz')
print(npzfile.files)
X_train = npzfile['arr_0']
y_train = npzfile['arr_1']
print(X_train.shape, y_train.shape)

['arr_0', 'arr_1']
(6394, 640, 128) (6394,)


In [0]:
npzfile = np.load('/content/gdrive/My Drive/Music-Genre-Classifier/Test Set/test_arr.npz')
print(npzfile.files)
X_test = npzfile['arr_0']
y_test = npzfile['arr_1']
print(X_test.shape, y_test.shape)

['arr_0', 'arr_1']
(800, 640, 128) (800,)


In [0]:
npzfile = np.load('/content/gdrive/My Drive/Music-Genre-Classifier/Validation Set/shuffled_valid.npz')
print(npzfile.files)
X_valid = npzfile['arr_0']
y_valid = npzfile['arr_1']
print(X_valid.shape, y_valid.shape)

['arr_0', 'arr_1']
(800, 640, 128) (800,)


In [0]:
y_train = tensorflow.keras.utils.to_categorical(y_train,8,dtype='float32')

In [0]:
y_valid = tensorflow.keras.utils.to_categorical(y_valid,8,dtype='float32')

In [0]:
print(y_train.shape,y_valid.shape)

(6394, 8) (800, 8)


In [0]:
# X_valid *=255
# X_test *=255
# X_train *=255

In [0]:
# X_test = np.squeeze(X_test,axis=-1)
X_valid = np.squeeze(X_valid,axis=-1)

In [0]:
print(X_train.shape,X_valid.shape,X_test.shape)

(6394, 640, 128) (800, 640, 128, 1) (800, 640, 128, 1)


In [0]:
# batch_size = 32
# num_classes = 8
# n_features = X_train.shape[2]
# n_time = X_train.shape[1]

In [0]:
# nb_filters1=16 
# nb_filters2=32 
# nb_filters3=64
# nb_filters4=64
# nb_filters5=64
# ksize = (3,1)
# pool_size_1= (2,2) 
# pool_size_2= (4,4)
# pool_size_3 = (4,2)

# dropout_prob = 0.20
# dense_size1 = 128
# lstm_count = 64
# num_units = 120

# BATCH_SIZE = 64
# EPOCH_COUNT = 50
# L2_regularization = 0.001

In [0]:
# def conv_recurrent_model_build(model_input):
#     print('Building model...')
#     layer = model_input
    
#     ### Convolutional blocks
#     conv_1 = Conv2D(filters = nb_filters1, kernel_size = ksize, strides=1,
#                       padding= 'valid', activation='relu', name='conv_1')(layer)
#     pool_1 = MaxPooling2D(pool_size_1)(conv_1)

#     conv_2 = Conv2D(filters = nb_filters2, kernel_size = ksize, strides=1,
#                       padding= 'valid', activation='relu', name='conv_2')(pool_1)
#     pool_2 = MaxPooling2D(pool_size_1)(conv_2)

#     conv_3 = Conv2D(filters = nb_filters3, kernel_size = ksize, strides=1,
#                       padding= 'valid', activation='relu', name='conv_3')(pool_2)
#     pool_3 = MaxPooling2D(pool_size_1)(conv_3)
    
    
#     conv_4 = Conv2D(filters = nb_filters4, kernel_size = ksize, strides=1,
#                       padding= 'valid', activation='relu', name='conv_4')(pool_3)
#     pool_4 = MaxPooling2D(pool_size_2)(conv_4)
    
    
#     conv_5 = Conv2D(filters = nb_filters5, kernel_size = ksize, strides=1,
#                       padding= 'valid', activation='relu', name='conv_5')(pool_4)
#     pool_5 = MaxPooling2D(pool_size_2)(conv_5)

#     flatten1 = Flatten()(pool_5)
#     ### Recurrent Block
    
#     # Pooling layer
#     pool_lstm1 = MaxPooling2D(pool_size_3, name = 'pool_lstm')(layer)
    
#     # Embedding layer

#     squeezed = Lambda(lambda x: K.squeeze(x, axis= -1))(pool_lstm1)
# #     flatten2 = K.squeeze(pool_lstm1, axis = -1)
# #     dense1 = Dense(dense_size1)(flatten)
    
#     # Bidirectional GRU
#     lstm = Bidirectional(GRU(lstm_count))(squeezed)  #default merge mode is concat
    
#     # Concat Output
#     concat = concatenate([flatten1, lstm], axis=-1, name ='concat')
    
#     ## Softmax Output
#     output = Dense(num_classes, activation = 'softmax', name='preds')(concat)
    
#     model_output = output
#     model = Model(model_input, model_output)
    
# #     opt = Adam(lr=0.001)
#     opt = RMSprop(lr=0.0005)  # Optimizer
#     model.compile(
#             loss='categorical_crossentropy',
#             optimizer=opt,
#             metrics=['accuracy']
#         )
    
#     print(model.summary())
#     return model

In [0]:
# def train_model(x_train, y_train, x_val, y_val):
    
#     n_frequency = 128
#     n_frames = 640
#     #reshape and expand dims for conv2d
# #     x_train = x_train.reshape(-1, n_frequency, n_frames)
#     x_train = np.expand_dims(x_train, axis = -1)
    
# #     x_val = x_val.reshape(-1, n_frequency, n_frames)
#     x_val = np.expand_dims(x_val, axis = -1)
    
    
#     input_shape = (n_frames, n_frequency, 1)
#     model_input = Input(input_shape, name='input')
    
#     model = conv_recurrent_model_build(model_input)
    
# #     tb_callback = TensorBoard(log_dir='./logs/4', histogram_freq=1, batch_size=32, write_graph=True, write_grads=False,
# #                               write_images=False, embeddings_freq=0, embeddings_layer_names=None,
# #                               embeddings_metadata=None)
#     checkpoint_callback = ModelCheckpoint('/content/gdrive/My Drive/Music-Genre-Classifier/Models/cnn_rnn_weights.best.h5', monitor='val_accuracy', verbose=1,
#                                           save_best_only=True, mode='max')
    
#     reducelr_callback = ReduceLROnPlateau(
#                 monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01,
#                 verbose=1
#             )
#     callbacks_list = [checkpoint_callback, reducelr_callback]

#     # Fit the model and get training history.
#     print('Training...')
#     history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCH_COUNT,
#                         validation_data=(x_val, y_val), verbose=1, callbacks=callbacks_list)

#     return model, history

In [0]:
# def show_summary_stats(history):
#     # List all data in history
#     print(history.history.keys())

#     # Summarize history for accuracy
#     plt.plot(history.history['accuracy'])
#     plt.plot(history.history['val_accuracy'])
#     plt.title('model accuracy')
#     plt.ylabel('accuracy')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()

#     # Summarize history for loss
#     plt.plot(history.history['loss'])
#     plt.plot(history.history['val_loss'])
#     plt.title('model loss')
#     plt.ylabel('loss')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()

In [0]:
# model, history  = train_model(X_train, y_train, X_valid, y_valid)

In [0]:
# show_summary_stats(history)

In [0]:
# reducelr_callback = ReduceLROnPlateau(
#                 monitor='val_accuracy', factor=0.5, patience=10, min_delta=0.01,
#                 verbose=1
#             )
# es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.01,verbose=1, patience=5)

callbacks_list = [reducelr_callback]
clf = ak.ImageClassifier(max_trials=5,loss='categorical_crossentropy',metrics=['accuracy'],objective='val_accuracy')
clf.fit(X_train,y_train,epochs = 3,validation_data=(X_valid,y_valid))

Train for 200 steps, validate for 25 steps
Epoch 1/3
200/200 [==============================] - 86s 432ms/step - loss: 2.7744 - accuracy: 0.2796 - val_loss: 1.7757 - val_accuracy: 0.3450
Epoch 2/3
200/200 [==============================] - 86s 428ms/step - loss: 1.3795 - accuracy: 0.5241 - val_loss: 1.7209 - val_accuracy: 0.4175
Epoch 3/3
200/200 [==============================] - 86s 428ms/step - loss: 0.7113 - accuracy: 0.7748 - val_loss: 2.1476 - val_accuracy: 0.3700


Train for 200 steps, validate for 25 steps
Epoch 1/3
200/200 [==============================] - 199s 993ms/step - loss: 1.7307 - accuracy: 0.3808 - val_loss: 14.6418 - val_accuracy: 0.1375
Epoch 2/3
200/200 [==============================] - 191s 956ms/step - loss: 1.5171 - accuracy: 0.4718 - val_loss: 3.6890 - val_accuracy: 0.1887
Epoch 3/3
200/200 [==============================] - 190s 952ms/step - loss: 1.4257 - accuracy: 0.5028 - val_loss: 13.5826 - val_accuracy: 0.1562


Train for 200 steps, validate for 25 steps
Epoch 1/3
200/200 [==============================] - 87s 433ms/step - loss: 2.7593 - accuracy: 0.2490 - val_loss: 1.7515 - val_accuracy: 0.3787
Epoch 2/3
200/200 [==============================] - 86s 429ms/step - loss: 1.4669 - accuracy: 0.4887 - val_loss: 1.6024 - val_accuracy: 0.4212
Epoch 3/3
200/200 [==============================] - 86s 428ms/step - loss: 0.8560 - accuracy: 0.7213 - val_loss: 2.0308 - val_accuracy: 0.3700


INFO:tensorflow:Oracle triggered exit
Train for 200 steps, validate for 25 steps
Epoch 1/3
200/200 [==============================] - 86s 431ms/step - loss: 3.1096 - accuracy: 0.2315 - val_loss: 1.8848 - val_accuracy: 0.2825
Epoch 2/3
200/200 [==============================] - 85s 427ms/step - loss: 1.4616 - accuracy: 0.4966 - val_loss: 1.7201 - val_accuracy: 0.4275
Epoch 3/3
200/200 [==============================] - 85s 427ms/step - loss: 0.8759 - accuracy: 0.7157 - val_loss: 2.1751 - val_accuracy: 0.4050


In [0]:
from sklearn.metrics import classification_report

y_true = np.argmax(y_valid, axis = 1)
X_valid = np.expand_dims(X_valid, axis = -1)
# y_pred = model.predict(X_valid)
y_pred = clf.predict(X_valid)
y_pred = np.argmax(y_pred, axis=1)
labels = [0,1,2,3,4,5,6,7]
target_names = dict_genres.keys()

print(y_true.shape, y_pred.shape)
print(classification_report(y_true, y_pred, target_names=target_names))

(800,) (800,)
               precision    recall  f1-score   support

   Electronic       0.39      0.41      0.40       100
 Experimental       0.37      0.36      0.36       100
         Folk       0.49      0.57      0.53       100
      Hip-Hop       0.58      0.47      0.52       100
 Instrumental       0.23      0.05      0.08       100
International       0.42      0.51      0.46       100
          Pop       0.21      0.33      0.26       100
         Rock       0.55      0.54      0.55       100

     accuracy                           0.41       800
    macro avg       0.40      0.41      0.39       800
 weighted avg       0.40      0.41      0.39       800



In [0]:
from tensorflow.keras.models import load_model

weights_path = '/content/gdrive/My Drive/Music-Genre-Classifier/Models/cnn_rnn_weights.best.h5'
model = load_model(weights_path)

In [0]:
# y_test -= 1
# print(np.amin(y_test), np.amax(y_test), np.mean(y_test))
print(y_test[:10])

[7 5 5 5 5 5 5 2 2 1]


In [0]:
X_test = np.squeeze(X_test,axis=-1)

In [0]:
from sklearn.metrics import classification_report

y_true = y_test
X_test = np.expand_dims(X_test, axis = -1)
y_pred = clf.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
labels = [0,1,2,3,4,5,6,7]
target_names = dict_genres.keys()

print(y_true.shape, y_pred.shape)
print(classification_report(y_true, y_pred, target_names=target_names))

(800,) (800,)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

   Electronic       0.00      0.00      0.00       100
 Experimental       0.16      0.48      0.24       100
         Folk       0.31      0.04      0.07       100
      Hip-Hop       0.00      0.00      0.00       100
 Instrumental       0.14      0.61      0.22       100
International       0.13      0.04      0.06       100
          Pop       0.12      0.01      0.02       100
         Rock       0.00      0.00      0.00       100

     accuracy                           0.15       800
    macro avg       0.11      0.15      0.08       800
 weighted avg       0.11      0.15      0.08       800



In [0]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

0.1475
